In [2]:
!pip install xgboost

You should consider upgrading via the 'C:\Users\user\anaconda3\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt

import xgboost as xgb

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(15) # Seed 고정

In [3]:
# Data Load

train_df = pd.read_csv('train.csv')


In [5]:
# 검사 합격 데이터 제거
train_df = train_df.drop(['X_04', 'X_23', 'X_47', 'X_48'], axis=1)

In [6]:
train_x = train_df.filter(regex='X') # Input : X Featrue : 56
train_y = train_df.filter(regex='Y') # Output : Y Feature : 14

In [8]:
# 피처 스케일링
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_data_scaled = scaler.fit_transform(train_x)

X_data_scaled.shape

(39607, 52)

In [23]:
# {'activation': 'relu', 'alpha': 0.0001, 'early_stopping': True, 'hidden_layer_sizes': 3, 'learning_rate': 'constant', 'learning_rate_init': 0.01, 'max_iter': 1000, 'power_t': 0.5, 'solver': 'adam', 'warm_start': False}

#learning_rate = 0.01 에서 0.0003 으로 바꿈


In [9]:
# MLPRegressor

from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor()
param_grid = {'hidden_layer_sizes': [i for i in range(2,20)],
              'activation': ['LeakyReLU', 'relu'],
              'solver': ['adam'],
              'learning_rate': ['constant'],
              'learning_rate_init': [0.0003],
              'power_t': [0.5],
              'alpha': [0.0001],
              'max_iter': [1000],
              'early_stopping': [True],
              'warm_start': [False]}
mlp_GS = GridSearchCV(mlp, param_grid=param_grid, 
                   cv=10, verbose=True, pre_dispatch='2*n_jobs')
mlp_GS.fit(train_x, train_y)


Fitting 10 folds for each of 36 candidates, totalling 360 fits


c:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\user\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py", line 673, in fit
    return self._fit(X, y, incremental=False)
  File "c:\Users\user\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py", line 357, in _fit
    self._validate_hyperparameters()
  File "c:\Users\user\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py", line 448, in _validate_hyperparameters
    raise ValueError("The activation '%s' is not supported. Supported "
ValueError: The activation 'LeakyReLU' is not su

GridSearchCV(cv=10, estimator=MLPRegressor(),
             param_grid={'activation': ['LeakyReLU', 'relu'], 'alpha': [0.0001],
                         'early_stopping': [True],
                         'hidden_layer_sizes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                                12, 13, 14, 15, 16, 17, 18,
                                                19],
                         'learning_rate': ['constant'],
                         'learning_rate_init': [0.0003], 'max_iter': [1000],
                         'power_t': [0.5], 'solver': ['adam'],
                         'warm_start': [False]},
             verbose=True)

In [20]:
df = pd.DataFrame(mlp_GS.cv_results_)
df = df.sort_values(by='mean_test_score', ascending=False)
print(df[['params', 'mean_test_score']])   

                                               params  mean_test_score
19  {'activation': 'relu', 'alpha': 0.0001, 'early...        -0.020649
18  {'activation': 'relu', 'alpha': 0.0001, 'early...        -0.078165
20  {'activation': 'relu', 'alpha': 0.0001, 'early...        -0.097494
21  {'activation': 'relu', 'alpha': 0.0001, 'early...        -0.199868
22  {'activation': 'relu', 'alpha': 0.0001, 'early...        -0.221935
23  {'activation': 'relu', 'alpha': 0.0001, 'early...        -0.227621
24  {'activation': 'relu', 'alpha': 0.0001, 'early...        -0.340907
25  {'activation': 'relu', 'alpha': 0.0001, 'early...        -0.518977
26  {'activation': 'relu', 'alpha': 0.0001, 'early...        -0.519520
27  {'activation': 'relu', 'alpha': 0.0001, 'early...        -0.555736
28  {'activation': 'relu', 'alpha': 0.0001, 'early...        -1.053244
30  {'activation': 'relu', 'alpha': 0.0001, 'early...        -1.071739
29  {'activation': 'relu', 'alpha': 0.0001, 'early...        -1.194001
31  {'

In [21]:
print(mlp_GS.best_params_)

{'activation': 'relu', 'alpha': 0.0001, 'early_stopping': True, 'hidden_layer_sizes': 3, 'learning_rate': 'constant', 'learning_rate_init': 0.0003, 'max_iter': 1000, 'power_t': 0.5, 'solver': 'adam', 'warm_start': False}


In [25]:
test_x = pd.read_csv('test.csv').drop(columns=['ID','X_04', 'X_23', 'X_47', 'X_48'])

In [26]:
preds = mlp_GS.predict(test_x)
print('Done.')

Done.


In [27]:
submit = pd.read_csv('sample_submission.csv')

In [28]:
for idx, col in enumerate(submit.columns):
    if col=='ID':
        continue
    submit[col] = preds[:,idx-1]
print('Done.')

Done.


In [30]:
submit.to_csv('mlp.csv', index=False)


In [38]:
mlp_GS.score

<bound method BaseSearchCV.score of GridSearchCV(cv=10, estimator=MLPRegressor(),
             param_grid={'activation': ['LeakyReLU', 'relu'], 'alpha': [0.0001],
                         'early_stopping': [True],
                         'hidden_layer_sizes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                                12, 13, 14, 15, 16, 17, 18,
                                                19],
                         'learning_rate': ['constant'],
                         'learning_rate_init': [0.0003], 'max_iter': [1000],
                         'power_t': [0.5], 'solver': ['adam'],
                         'warm_start': [False]},
             verbose=True)>